![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# Подготовка
## 1. import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max.columns', 100)

import re # for strings processing
import json
from pprint import pprint

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os_files = []
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        file = os.path.join(dirname, filename)
        print(file)
        os_files.append(file)

# Any results you write to the current directory are saved as output.

/kaggle/input/sf-dst-restaurant-rating/sample_submission.csv
/kaggle/input/sf-dst-restaurant-rating/main_task.csv
/kaggle/input/sf-dst-restaurant-rating/kaggle_task.csv
/kaggle/input/tripadviso-parsing/UNdata_Export_new.csv
/kaggle/input/tripadviso-parsing/country_data.json
/kaggle/input/tripadviso-parsing/currency_rates.json
/kaggle/input/tripadviso-parsing/data_scraping.py
/kaggle/input/tripadviso-parsing/parsed_data_output3.csv
/kaggle/input/tripadviso-parsing/UNdata_Export.csv


In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

## 2. Подгружаем внешние данные

In [4]:
# скрипт с загрузкой внешних данных - см draft.py на github

DATA_DIR = '/kaggle/input/'
countries_info = pd.read_json(DATA_DIR+'tripadviso-parsing/country_data.json', orient='records')
display(countries_info.sample(3))

# no Italy in this dataset
cities_info = pd.read_csv(DATA_DIR+'tripadviso-parsing/UNdata_Export_new.csv')
display(cities_info.sample(3))

with open(DATA_DIR+'tripadviso-parsing/currency_rates.json', 'r') as file:
    currency_rates = json.load(file)

pprint(currency_rates)

,name,topLevelDomain,alpha2Code,alpha3Code,callingCodes,capital,altSpellings,region,subregion,population,latlng,demonym,area,gini,timezones,borders,nativeName,numericCode,currencies,languages,translations,flag,regionalBlocs,cioc
30,Luxembourg,[.lu],LU,LUX,[352],Luxembourg,"[LU, Grand Duchy of Luxembourg, Grand-Duché de...",Europe,Western Europe,576200,"[49.75, 6.16666666]",Luxembourger,2586.0,30.8,[UTC+01:00],"[BEL, FRA, DEU]",Luxembourg,442.0,"[{'code': 'EUR', 'name': 'Euro', 'symbol': '€'}]","[{'iso639_1': 'fr', 'iso639_2': 'fra', 'name':...","{'de': 'Luxemburg', 'es': 'Luxemburgo', 'fr': ...",https://restcountries.eu/data/lux.svg,"[{'acronym': 'EU', 'name': 'European Union', '...",LUX
10,Czech Republic,[.cz],CZ,CZE,[420],Prague,"[CZ, Česká republika, Česko]",Europe,Eastern Europe,10558524,"[49.75, 15.5]",Czech,78865.0,26.0,[UTC+01:00],"[AUT, DEU, POL, SVK]",Česká republika,203.0,"[{'code': 'CZK', 'name': 'Czech koruna', 'symb...","[{'iso639_1': 'cs', 'iso639_2': 'ces', 'name':...","{'de': 'Tschechische Republik', 'es': 'Repúbli...",https://restcountries.eu/data/cze.svg,"[{'acronym': 'EU', 'name': 'European Union', '...",CZE
7,Bulgaria,[.bg],BG,BGR,[359],Sofia,"[BG, Republic of Bulgaria, Република България]",Europe,Eastern Europe,7153784,"[43.0, 25.0]",Bulgarian,110879.0,28.2,[UTC+02:00],"[GRC, MKD, ROU, SRB, TUR]",България,100.0,"[{'code': 'BGN', 'name': 'Bulgarian lev', 'sym...","[{'iso639_1': 'bg', 'iso639_2': 'bul', 'name':...","{'de': 'Bulgarien', 'es': 'Bulgaria', 'fr': 'B...",https://restcountries.eu/data/bgr.svg,"[{'acronym': 'EU', 'name': 'European Union', '...",BUL


,Country or Area,Year,Area,Sex,City,City type,Record Type,Reliability,Source Year,Value
746,Poland,2018,Total,Female,Kielce,City proper,Estimate - de jure,"Final figure, complete",2018.0,103130.00000
1003,Sweden,2007,Total,Male,Västerås,City proper,Estimate - de jure,"Final figure, complete",2008.0,66055.50000
187,France,2015,Total,Male,Nancy,City proper,Census - de jure - complete tabulation,"Final figure, complete",2018.0,50411.33591


{'base': 'EUR',
 'date': '2020-01-01',
 'historical': True,
 'rates': {'ALL': 122.45171,
           'BAM': 1.964199,
           'BGN': 1.956373,
           'BYN': 2.378735,
           'CHF': 1.085595,
           'CZK': 25.418639,
           'DKK': 7.470602,
           'EUR': 1,
           'GBP': 0.846759,
           'GIP': 0.911913,
           'HRK': 7.477257,
           'HUF': 330.939533,
           'ISK': 135.797181,
           'MDL': 19.394532,
           'MKD': 61.753104,
           'NOK': 9.843415,
           'PLN': 4.255381,
           'RON': 4.790322,
           'RSD': 117.623536,
           'RUB': 69.402014,
           'SEK': 10.483649,
           'UAH': 26.723859},
 'success': True,
 'timestamp': 1577923199}


In [5]:
countries_info['currency'] = countries_info['currencies'].astype('str').str.slice(start=11, stop=14)
countries_info['currency rate'] = countries_info['currency'].apply(lambda x: currency_rates['rates'][x])

In [6]:
# оставляем только данные, которым пока нашли применение (как пользоваться остальными, возм, придумаем позже):
countries_info = countries_info[['name', 'alpha3Code', 'capital', 'subregion', 'population', 'area', 'gini', 'borders', 'currency', 'currency rate']]
countries_info.sample(5)

,name,alpha3Code,capital,subregion,population,area,gini,borders,currency,currency rate
11,Denmark,DNK,Copenhagen,Northern Europe,5717014,43094.00,24.0,[DEU],DKK,7.470602
34,Monaco,MCO,Monaco,Western Europe,38400,2.02,NaN,[FRA],EUR,1.000000
26,Jersey,JEY,Saint Helier,Northern Europe,100800,116.00,NaN,[],GBP,0.846759
48,Svalbard and Jan Mayen,SJM,Longyearbyen,Northern Europe,2562,NaN,NaN,[],NOK,9.843415
19,Guernsey,GGY,St. Peter Port,Northern Europe,62999,78.00,NaN,[],GBP,0.846759


In [ ]:
# плохие внешние данные, города без перевода на англ

# for city in data['City'].str.lower().unique():
#     if city not in cities_info['City'].str.lower().unique():
#         print(city)
# np.sort(cities_info['City'].unique())

In [ ]:
# {'LISBOA': 'Lisbon', 'München': 'Munich', 'PRAHA': 'Prague', ''}

## 3. Функции для предобработки

# DATA

In [40]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
df_train.info()

In [ ]:
df_train.head(5)

In [ ]:
df_test.info()

In [ ]:
df_test.head(5)

In [ ]:
sample_submission.head(5)

In [ ]:
sample_submission.info()

In [35]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data.info()

In [9]:
print(df_train['Restaurant_id'].nunique(), 'уникальных из 40 000')
print(df_test['Restaurant_id'].nunique(), 'уникальных из 10 000')
print(data['Restaurant_id'].nunique(), 'уникальных из 50 000')
print('т.е. в трейне и тесте id повторяются')

11909 уникальных из 40 000
10000 уникальных из 10 000
13094 уникальных из 50 000
т.е. в трейне и тесте id повторяются


Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

In [ ]:
data.sample(5)

In [ ]:
data.Reviews[1]

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [99]:
# ранее мы спарсили данные по 3 колонкам напрямую с сайта ТА (см скрипт data_scraping.py). Заполним ими пропуски по возможности:
DATA_DIR = '/kaggle/input/tripadviso-parsing/'
parsed_data = pd.read_csv(DATA_DIR+'/parsed_data_output3.csv', index_col=0)
print(parsed_data.info())
parsed_data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39711 entries, 33 to 39884
Data columns (total 5 columns):
Restaurant_id        39711 non-null object
Cuisine Style        31798 non-null object
Price Range          29442 non-null object
Number of Reviews    35602 non-null float64
URL_TA               39711 non-null object
dtypes: float64(1), object(4)
memory usage: 1.8+ MB
None


,Restaurant_id,Cuisine Style,Price Range,Number of Reviews,URL_TA
33,id_2434,['Vietnamese'],NaN,4.0,/Restaurant_Review-g187265-d3543959-Reviews-Vi...
28,id_10057,NaN,NaN,4.0,/Restaurant_Review-g187147-d2040769-Reviews-Le...
7,id_825,['Italian'],NaN,9.0,/Restaurant_Review-g274924-d3199765-Reviews-Ri...
19,id_2791,"['Chinese', 'Asian']",$$ - $$$,18.0,/Restaurant_Review-g947638-d8155327-Reviews-Ho...
34,id_2427,"['Italian', 'Pizza', 'Fast Food']",$,77.0,/Restaurant_Review-g187791-d5263694-Reviews-Pi...


In [87]:
parsed_data.loc[parsed_data['Restaurant_id'] == 'id_1', 'Cuisine Style'].str.findall("'(.*?)'").explode().value_counts()

European               6
Indian                 2
Asian                  2
Vegetarian Friendly    2
Scandinavian           2
Mediterranean          2
British                1
Swedish                1
Scottish               1
Healthy                1
French                 1
International          1
Norwegian              1
Italian                1
Name: Cuisine Style, dtype: int64

In [90]:
cuisine_nans = data[data['Cuisine Style'].isna()]
cuisine_nans

,Restaurant_id,City,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating
7,id_7,Budapest,NaN,2330.0,NaN,NaN,"[[], []]",/Restaurant_Review-g274887-d8286886-Reviews-Be...,d8286886,0,0.0
10,id_10,Rome,NaN,1209.0,NaN,306.0,"[['Very pretentious, but drinks are top notch'...",/Restaurant_Review-g187791-d7818546-Reviews-Je...,d7818546,0,0.0
13,id_13,Hamburg,NaN,1000.0,NaN,30.0,"[['Surprisingly good pizza', 'Yummy'], ['09/28...",/Restaurant_Review-g187331-d1344523-Reviews-Il...,d1344523,0,0.0
18,id_18,Vienna,NaN,3360.0,NaN,2.0,"[[], []]",/Restaurant_Review-g190454-d8435085-Reviews-An...,d8435085,0,0.0
20,id_20,Barcelona,NaN,6085.0,NaN,8.0,"[[], []]",/Restaurant_Review-g187497-d8334752-Reviews-Re...,d8334752,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
49981,id_924,Lyon,NaN,925.0,NaN,29.0,"[[], []]",/Restaurant_Review-g187265-d5850306-Reviews-Re...,d5850306,1,4.0
49983,id_2487,Amsterdam,NaN,2494.0,NaN,2.0,"[['Italian food, as in Italy!'], ['05/24/2015']]",/Restaurant_Review-g188590-d8120959-Reviews-Sa...,d8120959,1,4.5
49986,id_7274,Madrid,NaN,7279.0,NaN,NaN,"[[], []]",/Restaurant_Review-g187514-d12182212-Reviews-C...,d12182212,1,3.0
49988,id_4968,Berlin,NaN,4970.0,NaN,NaN,"[[], []]",/Restaurant_Review-g187323-d7761701-Reviews-Ar...,d7761701,1,4.0


In [169]:
print(parsed_data.loc[39853]['URL_TA'])
parsed_data.loc[parsed_data['URL_TA'].isin(cuisine_nans['URL_TA'])].dropna()

/Restaurant_Review-g187147-d10248508-Reviews-Jais-Paris_Ile_de_France.html


,Restaurant_id,Cuisine Style,Price Range,Number of Reviews,URL_TA
1,id_1535,"['Asian', 'Nepali']",$$ - $$$,15.0,/Restaurant_Review-g189852-d7992032-Reviews-Bu...
5,id_1418,"['Bar', 'European', 'Portuguese']",$$ - $$$,45.0,/Restaurant_Review-g189180-d12503536-Reviews-D...
10,id_6578,"['Mediterranean', 'Spanish']",$,12.0,/Restaurant_Review-g187497-d10696479-Reviews-R...
26,id_2763,"['Spanish', 'Wine Bar']",$$ - $$$,11.0,/Restaurant_Review-g187514-d10060659-Reviews-G...
90,id_499,"['French', 'Vegetarian Friendly', 'Vegan Optio...",$,55.0,/Restaurant_Review-g187265-d7623654-Reviews-Cr...
...,...,...,...,...,...
39853,id_3239,"['French', 'European']",$$$$,85.0,/Restaurant_Review-g187147-d10248508-Reviews-J...
39823,id_5087,"['French', 'Vegan Options']",$$ - $$$,109.0,/Restaurant_Review-g187147-d12179082-Reviews-M...
39947,id_276,"['International', 'European']",$$ - $$$,9.0,/Restaurant_Review-g274924-d4769868-Reviews-St...
39993,id_6057,['Dessert'],$$ - $$$,28.0,/Restaurant_Review-g187147-d10532509-Reviews-B...


In [170]:
data.loc[(data['Restaurant_id'] == 'id_3239') & (data['URL_TA'] == '/Restaurant_Review-g187147-d10248508-Reviews-Jais-Paris_Ile_de_France.html')]
# data.loc[(data['URL_TA'] == '/Restaurant_Review-g187147-d1912643-Reviews-R_Yves-Paris_Ile_de_France.html')]

,Restaurant_id,City,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating
49853,id_3239,Paris,NaN,3240.0,NaN,43.0,"[['Deliciousness', 'Romantic and delicious'], ...",/Restaurant_Review-g187147-d10248508-Reviews-J...,d10248508,1,4.5


In [175]:
parsed_data.loc[(parsed_data['Restaurant_id'] == 'id_3239') & (parsed_data['URL_TA'] == '/Restaurant_Review-g187147-d10248508-Reviews-Jais-Paris_Ile_de_France.html'), 'Cuisine Style']

39853    ['French', 'European']
Name: Cuisine Style, dtype: object

In [183]:
def fillna_parsed(row, obj):
    if pd.isna(row[obj]):
        rest_id = row['Restaurant_id']
        rest_url = row['URL_TA']
        parsed_values = parsed_data.loc[(parsed_data['Restaurant_id'] == rest_id) & (parsed_data['URL_TA'] == rest_url), obj]
        print(rest_id, rest_url, parsed_values.shape[0]) if parsed_values.shape[0] > 1 else None
        return parsed_values.iloc[0] if parsed_values.shape[0] == 1 else None
    else:
        return row[obj]


# mn = data.apply(fillna_parsed, obj='Cuisine Style', axis=1)

In [177]:
# mn[mn.notna()]

0                                           ['Bar', 'Pub']
1        ['European', 'Scandinavian', 'Gluten Free Opti...
2                                  ['Vegetarian Friendly']
3        ['Italian', 'Mediterranean', 'European', 'Vege...
4        ['Italian', 'Mediterranean', 'European', 'Seaf...
                               ...                        
49995    ['Italian', 'Vegetarian Friendly', 'Vegan Opti...
49996    ['French', 'American', 'Bar', 'European', 'Veg...
49997                                ['Japanese', 'Sushi']
49998    ['Polish', 'European', 'Eastern European', 'Ce...
49999                                          ['Spanish']
Length: 43010, dtype: object

In [11]:
def fillna_parsed_old(row, obj):
    if pd.isna(row[obj]):
        rest_id = row['Restaurant_id']
        parsed_values = parsed_data.loc[parsed_data['Restaurant_id'] == rest_id, obj]
        if parsed_values.shape[0] > 1 and any(parsed_values.notna()):   # больше 1 и есть не NaN значения
            return (parsed_values.loc[parsed_values.str.len().idxmax()] if obj == 'Cuisine Style'    # пока выбираем самую длинную строчку с кухнями
                                                                            # можно поменять на самые встречающиеся кухни из полученного набора для данного id
                    else parsed_values.value_counts().idxmax() if obj == 'Price Range'    # most common value
                    else parsed_values.median())    # median of numbers of reviews
        
        elif parsed_values.shape[0] == 0:    # если нет спарсенных данных по этому ресторану
            return np.nan
        else:
            return parsed_values.iloc[0]    # всего одно значение / одно или больше NaN значений - берем первое
        
    else:
        return row[obj]


# for col in ['Cuisine Style', 'Price Range', 'Number of Reviews']:
#     data[col] = data.apply(fillna_parsed, obj=col, axis=1)
#     print(f'{col} finished')

# print(data.info())
# data.sample(3)

In [12]:
# Для примера я возьму столбец Number of Reviews + Price Range
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')
data['Price_Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')
data[['Number_of_Reviews_isNAN', 'Price_Range_isNAN']].sample(5)

,Number_of_Reviews_isNAN,Price_Range_isNAN
29187,0,0
39868,0,0
43206,1,0
44000,0,0
40553,0,0


In [13]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(0, inplace=True)

In [14]:
# пропуски в Price Range заполним самым встречающимся по городу:

city_ranges = data['City'].value_counts()

for city in city_ranges.index:
    city_ranges[city] = data[data['City'] == city]['Price Range'].value_counts().idxmax()

print(city_ranges)

London        $$ - $$$
Paris         $$ - $$$
Madrid        $$ - $$$
Barcelona     $$ - $$$
Berlin        $$ - $$$
Milan         $$ - $$$
Rome          $$ - $$$
Prague        $$ - $$$
Lisbon        $$ - $$$
Vienna        $$ - $$$
Amsterdam     $$ - $$$
Brussels      $$ - $$$
Hamburg       $$ - $$$
Munich        $$ - $$$
Lyon          $$ - $$$
Stockholm     $$ - $$$
Budapest      $$ - $$$
Warsaw        $$ - $$$
Dublin        $$ - $$$
Copenhagen    $$ - $$$
Athens        $$ - $$$
Edinburgh     $$ - $$$
Zurich        $$ - $$$
Oporto        $$ - $$$
Geneva        $$ - $$$
Krakow        $$ - $$$
Oslo          $$ - $$$
Helsinki      $$ - $$$
Bratislava    $$ - $$$
Luxembourg    $$ - $$$
Ljubljana     $$ - $$$
Name: City, dtype: object


In [15]:
# выше видно, что среднее по всем городам одинаковое, поэтому пропуски заполним константой:
data['Price Range'].fillna('$$ - $$$', inplace=True)

In [ ]:
data.info()

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [16]:
data.nunique(dropna=False)

Restaurant_id              13094
City                          31
Cuisine Style              10732
Ranking                    12975
Price Range                    3
Number of Reviews           1574
Reviews                    41858
URL_TA                     49963
ID_TA                      49963
sample                         2
Rating                        10
Number_of_Reviews_isNAN        2
Price_Range_isNAN              2
dtype: int64

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.
Не будем сейчас сильно погружаться в эту тематику, давайте посмотрим лучше пример с One-Hot Encoding:
![](https://i.imgur.com/mtimFxh.png)

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=['City'], dummy_na=True)

In [ ]:
data.sample(5)

#### Возьмем следующий признак "Price Range".

In [ ]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3  
*Попробуйте сделать обработку этого признака уже самостоятельно!*

In [ ]:
price_ranges = {np.nan: 0, '$': 1, '$$ - $$$': 2, '$$$$': 3}
data['Price Range'] = data['Price Range'].apply(lambda x: price_ranges[x])

In [ ]:
data['Price Range'].describe()

> Для некоторых алгоритмов МЛ даже для не категориальных признаков можно применить One-Hot Encoding, и это может улучшить качество модели. Пробуйте разные подходы к кодированию признака - никто не знает заранее, что может взлететь.

### Обработать другие признаки вы должны самостоятельно!
Для обработки других признаков вам возможно придется даже написать свою функцию, а может даже и не одну, но в этом и есть ваша практика в этом модуле!     
Следуя подсказкам в модуле вы сможете более подробно узнать, как сделать эти приобразования.

In [ ]:
# тут ваш код на обработку других признаков
# .....

![](https://cs10.pikabu.ru/post_img/2018/09/06/11/1536261023140110012.jpg)

1. Cuisine Style

In [17]:
def cuisine_notna_dummies(styles, current):
    return (0 if pd.isnull(styles)
            else int(current in styles))
    
    
def cuisine_dummies(data):
    pat = re.compile('\'(.*?)\'')
    cuisine_styles = data['Cuisine Style'].dropna().str.findall('\'(.*?)\'')
    unique_cuisine_styles = np.sort(cuisine_styles.explode().unique())

    for cuisine in unique_cuisine_styles:
        cuisine_prefixed = f'Cuisine_{cuisine}'
        data[cuisine_prefixed] = data['Cuisine Style'].apply(cuisine_notna_dummies, current=cuisine)

    data['Cuisine_nan'] = pd.isna(data['Cuisine Style']).astype('uint8')   # if cuisines in strings
    
    return data

# data = cuisine_dummies(data)
# data.sample(3)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

### Adding features

In [ ]:
# 1. Is in a capital?
# data['is_capital'] = data['City'].isin(countries_info['capital'].values).astype('uint8')

In [ ]:
# 2. Weighed ranking

# cities_max_rank = data.groupby('City')['Ranking'].max()


def get_weighed_rank(row, obj, rank_table):
    city = row['City']
    value = row[obj]
    return value / rank_table.loc[city]


# data.apply(get_weighed_rank, axis=1)

In [ ]:
# 3. Gini index

# countries_info[countries_info['capital'].isin(data['City'].unique())]
# cities_info

In [ ]:
# for city in data['City'].str.lower().unique():
#     if city not in cities_info['City'].str.lower().unique():
#         print(city)
# np.sort(cities_info['City'].unique())

In [ ]:
# 4. Cuisine styles number
data['Cuisine_styles_number'] = data['Cuisine Style'].str.findall('\'(.*?)\'').str.len()

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [18]:
data

,Restaurant_id,City,Cuisine Style,Ranking,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,sample,Rating,Number_of_Reviews_isNAN,Price_Range_isNAN
0,id_0,Paris,"['Bar', 'Pub']",12963.0,$$ - $$$,4.0,"[[], []]",/Restaurant_Review-g187147-d10746918-Reviews-L...,d10746918,0,0.0,0,0
1,id_1,Helsinki,"['European', 'Scandinavian', 'Gluten Free Opti...",106.0,$$ - $$$,97.0,"[['Very good reviews!', 'Fine dining in Hakani...",/Restaurant_Review-g189934-d6674944-Reviews-Ra...,d6674944,0,0.0,0,0
2,id_2,Edinburgh,['Vegetarian Friendly'],810.0,$$ - $$$,28.0,"[['Better than the Links', 'Ivy Black'], ['12/...",/Restaurant_Review-g186525-d13129638-Reviews-B...,d13129638,0,0.0,0,0
3,id_3,London,"['Italian', 'Mediterranean', 'European', 'Vege...",1669.0,$$$$,202.0,"[['Most exquisite', 'Delicious and authentic']...",/Restaurant_Review-g186338-d680417-Reviews-Qui...,d680417,0,0.0,0,0
4,id_4,Bratislava,"['Italian', 'Mediterranean', 'European', 'Seaf...",37.0,$$$$,162.0,"[['Always the best in bratislava', 'Very good ...",/Restaurant_Review-g274924-d1112354-Reviews-Ma...,d1112354,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,1,4.5,0,0
49996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,1,3.5,0,0
49997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,$$ - $$$,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,1,4.5,0,1
49998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,1,4.0,0,0


In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
#     df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    df_output.drop(['ID_TA'], axis = 1, inplace=True)
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    # тут ваш код по обработке NAN
    print("Filling in NaNs in 'Cuisine Style', 'Price Range', 'Number of Reviews' with parsed data...")
    
    for col in ['Cuisine Style', 'Price Range', 'Number of Reviews']:
#     for col in ['Cuisine Style', 'Number of Reviews']:
        df_output[col] = df_output.apply(fillna_parsed, obj=col, axis=1)
        print(f'{col} finished')
    
    print('\nMarking still missing values...\n')
    df_output['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')
    df_output['Price_Range_isNAN'] = pd.isna(df_output['Price Range']).astype('uint8')
    
    print('Filling them in...\n')
    df_output['Number of Reviews'].fillna(0, inplace=True)
    df_output['Price Range'].fillna('$$ - $$$', inplace=True)
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    print('Encoding features...\n')
#     df_output = pd.get_dummies(df_output, columns=['City'], dummy_na=True)
    # тут ваш код не Encoding фитчей
    
    df_output = cuisine_dummies(df_output)
    
    price_ranges = {np.nan: 0, '$': 1, '$$ - $$$': 2, '$$$$': 3}
    df_output['Price Range'] = df_output['Price Range'].replace(to_replace=price_ranges)
    
    
    # ################### 4. Feature Engineering ####################################################
    # тут ваш код не генерацию новых фитчей
    print('Feature engineering...\n')
    # 1. Is in a capital?
    df_output['is_capital'] = df_output['City'].isin(countries_info['capital'].values).astype('uint8')
    
    # 2. Weighed ranking
#     cities_max_rank = df_output.groupby('City')['Ranking'].max()
    rest_count = df_output.groupby('City')['Restaurant_id'].count()
    df_output['Weighed_ranking'] = df_output.apply(get_weighed_rank, obj='Ranking', rank_table=rest_count, axis=1)
    
    # 3. Weighed number of reviews
#     cities_max_rev = df_output.groupby('City')['Number of Reviews'].max()
    df_output['Weighed_reviews_num'] = df_output.apply(get_weighed_rank, obj='Number of Reviews', rank_table=rest_count, axis=1)
    
    # 4. Cuisine styles number
    df_output['Cuisine_styles_number'] = df_output['Cuisine Style'].str.findall('\'(.*?)\'').str.len()
    df_output['Cuisine_styles_number'].fillna(1, inplace=True)
    
#     city dummies
    df_output = pd.get_dummies(df_output, columns=['City'], dummy_na=True)
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
data

In [ ]:
df_preproc.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
